# SQL analysis part 1

Download this Juypter Notebook and solve the tasks by inserting the SQL queries (it is not possible to solve the tasks in Colab). 

Example query (we include `df_example` at the end of the code cell to print the result):

```Python
df_example = pd.read_sql("""
    SELECT *
    FROM ecommerce_data;
""", engine)

df_example
```

## Setup

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

## Data

Connect to your MySQL-database "db_ecommerce" (make sure to prepare your `.env` file)

In [2]:
load_dotenv()   # take environment variables from .env

engine = create_engine("mysql+pymysql://" + os.environ['DB_URL'] + "/db_ecommerce", pool_pre_ping=True, pool_recycle=300)

In [3]:
# Use pandas to_sql function to create the table in the database
df = pd.read_csv('https://raw.githubusercontent.com/kirenz/lab-competitive/main/code/ecommerce.csv')
df.to_sql('ecommerce', engine, if_exists='replace')

108


## Task 1  

Get the list of distinct E-shops that have annual revenue more than 70000 and less than 75000 (note that annual revenue is recorded in 10K, i.e. 70 in the data equals 70000)


In [17]:
df1 = pd.read_sql("""
    SELECT DISTINCT eshop_name
    FROM ecommerce_data
    WHERE annual_revenue BETWEEN 70 AND 75
    ORDER BY eshop_name;
""", engine)

df1


,eshop_name
0,E-ShopA
1,E-ShopB


## Task 2

Get the top 3 records with the highest annual revenue and which have 'b' or 'a' in their names (note that you need to use two % in Python, e.g. '%%a%%' instead of '%a%'). Show the annual revenue in thousands of dollars (e.g. 10000 instead of 10) and name it revenue.

In [30]:
df2 = pd.read_sql("""
    SELECT eshop_name, annual_revenue * 1000 AS revenue
    FROM ecommerce_data
    WHERE eshop_name LIKE '%%a%%' OR eshop_name LIKE '%%b%%'
    ORDER BY revenue DESC
    LIMIT 3;
""", engine)

df2


,eshop_name,revenue
0,E-ShopA,74670.0
1,E-ShopB,70840.0
2,E-ShopA,66270.0


## Task 3

Get the list of E-shops that have `NULL` in social_media_followers

In [27]:
df3 = pd.read_sql("""
    SELECT eshop_name
    FROM ecommerce_data
    WHERE social_media_followers IS NULL;
""", engine)

df3


,eshop_name


## Task 4

Get the E-shops whose average rating is in (3.5, 4.0, 4.5). Note that only returns values which exactly matches one of the conditions (this is not an intervall).

In [49]:
df4 = pd.read_sql("""
    SELECT eshop_name, AVG(average_rating) as average_rating
    FROM ecommerce_data
    WHERE average_rating IN (3.5, 4.5, 5.0) 
    GROUP BY eshop_name
    ORDER BY average_rating DESC;
""", engine)

df4


,eshop_name,average_rating
0,E-ShopA,4.5


## Task 5

Get the E-shops with social media followers that are above 1400 and have 'a' or 'c' in their names. Use REGEXP to obtain the result.

In [58]:
df5 = pd.read_sql("""
    SELECT eshop_name, social_media_followers
    FROM ecommerce_data
    WHERE eshop_name REGEXP 'a|c' AND social_media_followers > 1400
    ORDER BY social_media_followers DESC;
""", engine)

df5


,eshop_name,social_media_followers
0,E-ShopC,1529.19
1,E-ShopA,1417.39


## Task 6

Get the E-shops with annual revenue above 70000 or with social media followers that are above 1400 and end with 'a' in their names. Use REGEXP to obtain the result.

In [60]:
df6 = pd.read_sql("""
    SELECT eshop_name, social_media_followers, annual_revenue
    FROM ecommerce_data
    WHERE annual_revenue > 70 OR
        (eshop_name REGEXP 'a$' AND social_media_followers > 1400)
    ORDER BY social_media_followers DESC;
""", engine)


df6


,eshop_name,social_media_followers,annual_revenue
0,E-ShopA,1417.39,74.67
1,E-ShopB,1212.05,70.84


## Close the connection

In [28]:
# close connection
engine.dispose()